#NIH Chest Xray Data Exploration



This dataset is know to have poor data integrity. New folders will be created to review, re-interpreted, and relabel the images. These new dataset will be then be used to train an simple CNN model.

#Import Libraries 

In [ ]:
import pandas as pd
import os
import shutil
import splitfolders
import random

#Data Exploration

In [ ]:
# create file_path and load CSV file
Images_PATH= "Documents/ML/CXR"
df= pd.read_csv("Data_Entry_2017.csv")
pd.set_option('display.max_rows', 100)
df


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,30801,39,M,PA,2048,2500,0.168,0.168,NaN
112116,00030802_000.png,No Finding,0,30802,29,M,PA,2048,2500,0.168,0.168,NaN
112117,00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,0.168,NaN
112118,00030804_000.png,No Finding,0,30804,30,F,PA,2048,2500,0.168,0.168,NaN


Select colomns.

In [ ]:
df= df[['Image Index','Finding Labels', 'Follow-up #', 'Patient Age','Patient Gender', 'View Position' ]]
df

,Image Index,Finding Labels,Follow-up #,Patient Age,Patient Gender,View Position
0,00000001_000.png,Cardiomegaly,0,58,M,PA
1,00000001_001.png,Cardiomegaly|Emphysema,1,58,M,PA
2,00000001_002.png,Cardiomegaly|Effusion,2,58,M,PA
3,00000002_000.png,No Finding,0,81,M,PA
4,00000003_000.png,Hernia,0,81,F,PA
...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,39,M,PA
112116,00030802_000.png,No Finding,0,29,M,PA
112117,00030803_000.png,No Finding,0,42,F,PA
112118,00030804_000.png,No Finding,0,30,F,PA


Label categories and counts.

In [ ]:
#Data exploration of finding labels
df['Finding Labels'].value_counts()

No Finding                                                                60361
Infiltration                                                               9547
Atelectasis                                                                4215
Effusion                                                                   3955
Nodule                                                                     2705
                                                                          ...  
Atelectasis|Consolidation|Infiltration|Pleural_Thickening|Pneumothorax        1
Atelectasis|Emphysema|Fibrosis|Mass                                           1
Edema|Infiltration|Mass|Pneumothorax                                          1
Atelectasis|Cardiomegaly|Consolidation|Edema|Effusion|Mass                    1
Edema|Effusion|Pleural_Thickening                                             1
Name: Finding Labels, Length: 836, dtype: int64

In clinical practice young adults have cleaner appearing chest xrays (with less chronic changes) and acute disease changes are prominent. Images with ages between 20-40 and PA view will be selected for new dataframe and new folders will be created from this dataframe.

In [ ]:
# Create and view new dataframe
adult_20_40 = df[(df['Patient Age']>= 20) & (df['Patient Age'] <=40)]
adult_20_40.head(10)

,Image Index,Finding Labels,Follow-up #,Patient Age,Patient Gender,View Position
407,00000101_000.png,Nodule,0,33,M,PA
573,00000140_000.png,No Finding,0,25,F,PA
574,00000141_000.png,No Finding,0,31,M,PA
630,00000157_000.png,Nodule,0,40,F,PA
777,00000190_000.png,No Finding,0,30,M,PA
778,00000190_001.png,No Finding,1,31,M,PA
779,00000190_002.png,No Finding,2,33,M,PA
780,00000190_003.png,No Finding,3,32,M,PA
781,00000190_004.png,No Finding,4,32,M,PA
782,00000190_005.png,No Finding,5,34,M,PA


Find number of studies, any NaN and counts of respective finding labels.

In [ ]:

adult_20_40.count()

Image Index       31131
Finding Labels    31131
Follow-up #       31131
Patient Age       31131
Patient Gender    31131
View Position     31131
dtype: int64

In [ ]:
adult_20_40.isnull().sum()

Image Index       0
Finding Labels    0
Follow-up #       0
Patient Age       0
Patient Gender    0
View Position     0
dtype: int64

In [ ]:
adult_20_40['Finding Labels'].value_counts()

No Finding                                        18006
Infiltration                                       2965
Effusion                                            881
Atelectasis                                         833
Pneumothorax                                        641
                                                  ...  
Atelectasis|Edema|Effusion|Infiltration|Mass          1
Fibrosis|Nodule|Pneumothorax                          1
Consolidation|Effusion|Mass|Pleural_Thickening        1
Consolidation|Edema|Effusion|Infiltration             1
Atelectasis|Edema|Infiltration|Pneumonia              1
Name: Finding Labels, Length: 459, dtype: int64

Find the count for the label "No Finding" and view PA.

In [ ]:
cnt= 0 

for (i, row) in adult_20_40.iterrows():
    if row["Finding Labels"]== "No Finding" and row['View Position']== 'PA':
        cnt += 1
print(cnt)

Create folder for images with labels "No Finding" and "PA" view for review. 

In [ ]:
TARGET_DIR= 'Dataset/Normal'

if not os.path.exists(TARGET_DIR):
    os.mkdir(TARGET_DIR)
    print("CXR folder created")

In [ ]:
#Exporting images into new folder
cnt= 0 

for (i, row) in adult_20_40.iterrows():
    if row["Finding Labels"]== "No Finding" and row['View Position']== 'PA':
        Image_Index= row['Image Index']
        image_path= os.path.join(Images_PATH, Image_Index)
        image_copy_path = os.path.join(TARGET_DIR, Image_Index) 
        shutil.copy2(image_path, image_copy_path)
        print("moving image", cnt)
        cnt += 1

Create image folder with infiltration as positive finding.

In [ ]:
#Exporting images into new folder
cnt= 0 

for (i, row) in adult_20_40.iterrows():
    if row["Finding Labels"]== "Infiltration" and row['View Position']== 'PA':
        cnt += 1
print(cnt)

In [ ]:
#Create image folder with infiltrate as positive finding
TARGET_DIR= 'Dataset/Positive'

if not os.path.exists(TARGET_DIR):
    os.mkdir(TARGET_DIR)
    print("CXR folder created")

In [ ]:
cnt= 0 

for (i, row) in adult_20_40.iterrows():
    if row["Finding Labels"]== "Infiltration" and row['View Position']== 'PA':
        Image_Index= row['Image Index']
        image_path= os.path.join(Images_PATH, Image_Index)
        image_copy_path = os.path.join(TARGET_DIR, Image_Index) 
        shutil.copy2(image_path, image_copy_path)
        print("moving image", cnt)
        cnt += 1

#Summary of the Image Quality


    

The "No Finding" folder had poor data integrity. There a significant number of films mislabeled (positive findings instead of no findings); included both AP and lateral; inaccurate ages bases on xray findings (skeletally immature or advanced degenerative changes with bone demineralization); poor diagnostic quality of the study. The images were re-read and placed in a new folder was create labeled Negative_clean.

The "Infiltrate" file again consisted of poor data integrity. Majority of images had no pulmonary findings. The images were re-read and plaed in a new folder labeled Positive_clean. Since only a small percentage could be used, more folders were created with "Consolidation" and "Pneumonia" labels used for selection.

#Creating New Chest X-ray Image Dataset

Additional finding label were use to increase the number of positive findings. Consolidation, pneunomia and infiltrate present similarly in chest x-ray. Although pneunomia is techincally a diagnosis and not a strict finding, the presence of consolidation or inflitrate lead to a diagnosis of pneumonia. Positive chest x-rays in the no finding folder were manually moved into the positive dataset. 

In [ ]:
TARGET_DIR= 'Dataset/Positive_consolidation'

if not os.path.exists(TARGET_DIR):
    os.mkdir(TARGET_DIR)
    print("CXR folder created")

In [ ]:
cnt= 0 

for (i, row) in adult_20_40.iterrows():
    if row["Finding Labels"]== "Consolidation" and row['View Position']== 'PA':
        Image_Index= row['Image Index']
        image_path= os.path.join(Images_PATH, Image_Index)
        image_copy_path = os.path.join(TARGET_DIR, Image_Index) 
        shutil.copy2(image_path, image_copy_path)
        print("moving image", cnt)
        cnt += 1

In [ ]:
TARGET_DIR= 'Dataset/Positive_pneumonia'

if not os.path.exists(TARGET_DIR):
    os.mkdir(TARGET_DIR)
    print("CXR folder created")

In [ ]:
cnt= 0 

for (i, row) in adult_20_40.iterrows():
    if row["Finding Labels"]== "Pneumonia" and row['View Position']== 'PA':
        Image_Index= row['Image Index']
        image_path= os.path.join(Images_PATH, Image_Index)
        image_copy_path = os.path.join(TARGET_DIR, Image_Index) 
        shutil.copy2(image_path, image_copy_path)
        print("moving image", cnt)
        cnt += 1

In [ ]:
TARGET_DIR= 'Dataset/Negative_clean'

if not os.path.exists(TARGET_DIR):
    os.mkdir(TARGET_DIR)
    print("CXR folder created")

In [ ]:
TARGET_DIR= 'Dataset/Negative_clean'

if not os.path.exists(TARGET_DIR):
    os.mkdir(TARGET_DIR)
    print("CXR folder created")

In [ ]:
#Create smaller dataset of cleaned no findings
files_list = []

for root, dirs, files in os.walk(r"C:\Users\Matrix BR\Dataset\Negative_clean"):
    for file in files:
        if  file.endswith(".png"):
            files_list.append(os.path.join(root, file))

files_normal = random.sample(files_list, 1200) 

destPath = r"C:\Users\Matrix BR\Dataset\Clean\Negative"  

if os.path.isdir(destPath) == False:
        os.makedirs(destPath)

# iteraate over all random files and copy them
for file in files_normal:
    shutil.copy(file, destPath)

print(len(files_normal))

In [ ]:
Negative_images = df[(df["Image Index"].isin(file_CXR))]

Create new CSV files:

In [ ]:
Negative_images.count()

In [ ]:
Negative_images['Finding Labels'].value_counts()


In [ ]:
Negative_CXR = Negative_images.replace(['Infiltration','No Finding'], 'Negative')
Negative_CXR['Finding Labels'].value_counts()

In [ ]:
#create new csv
Negative_CXR.to_csv('Negative_clean.csv')

In [ ]:
Negative_CXR= pd.read_csv("Negative_clean.csv")
Negative_CXR

In [ ]:
TARGET_DIR= 'Dataset/Positive_clean'

if not os.path.exists(TARGET_DIR):
    os.mkdir(TARGET_DIR)
    print("CXR folder created")

In [ ]:
Positive_images = 'Dataset/Positive_clean'
file_CXR = os.listdir(Positive_images)


In [ ]:
Positive_clean = df[(df["Image Index"].isin(file_CXR))]

In [ ]:
Positive_clean.count()

In [ ]:
Positive_clean['Finding Labels'].value_counts()

In [ ]:
Positive_clean = Positive_clean.replace(['Infiltration','No Finding', 'Consolidation', 'Pneumonia'], 'Positive')
Positive_clean['Finding Labels'].value_counts()

In [ ]:
Positive_clean.to_csv('Positive_clean.csv')

In [ ]:
Positive_clean

Create a new dataset folder of uncleaned images of positive and negative, "Junk" images.

In [ ]:
# Create new file of re-read xrays of dirty positive findings
TARGET_DIR= 'Dataset/Junk_positive'

if not os.path.exists(TARGET_DIR):
    os.mkdir(TARGET_DIR)
    print("CXR folder created")

In [ ]:
cnt=0

for (i, row) in adult_20_40.iterrows():
    if row["Finding Labels"]== "Infiltration" and row['View Position']== 'PA':
        Image_Index= row['Image Index']
        image_path= os.path.join(Images_PATH, Image_Index)
        image_copy_path= os.path.join(TARGET_DIR, Image_Index)
        shutil.copy2(image_path, image_copy_path)
        print("moving image", cnt)
        cnt+= 1

Create negative dataset folder by randomly selecting 1555 images from the uncleaned no finding label to match the number of infiltrate images, 1555, for the "Junk" dataset. 

In [ ]:
TARGET_DIR= 'Dataset/Junk_negative'

if not os.path.exists(TARGET_DIR):
    os.mkdir(TARGET_DIR)
    print("CXR folder created")

In [ ]:
#Create smaller dataset of non cleaned no findings
files_list = []

for root, dirs, files in os.walk(r"C:\Users\Matrix BR\Dataset\Normal"):
    for file in files:
        if  file.endswith(".png"):
            files_list.append(os.path.join(root, file))

files_normal = random.sample(files_list, 1555) 

destPath = r"C:\Users\Matrix BR\Dataset\Junk_images\Junk_negative"  

if os.path.isdir(destPath) == False:
        os.makedirs(destPath)

# iteraate over all random files and move them
for file in files_normal:
    shutil.copy(file, destPath)
    
print(len(files_normal))

Larger dataset to be sorted by the classifier. The complete infiltrate, consolidation, pneumonia folders in the original dataset were combined and used for the positive. 

In [ ]:
TARGET_DIR= 'Dataset/Negative_for_Sorter'

if not os.path.exists(TARGET_DIR):
    os.mkdir(TARGET_DIR)
    print("CXR folder created")

Create dataset of "Junk" no finding images to be resorted through the calssifier.

In [ ]:
#Exporting images into new folder
cnt= 0 

for (i, row) in adult_20_40.iterrows():
    if row["Finding Labels"]== "No Finding" and row['View Position']== 'PA':
        Image_Index= row['Image Index']
        image_path= os.path.join(Images_PATH, Image_Index)
        image_copy_path = os.path.join(TARGET_DIR, Image_Index) 
        shutil.copy2(image_path, image_copy_path)
        print("moving image", cnt)
        cnt += 1

In [ ]:
#Create smaller 10,000 image dataset of no findings
files_list = []

for root, dirs, files in os.walk(r"C:\Users\Matrix BR\Dataset\Negative_for_Sorter"):
    for file in files:
        if  file.endswith(".png"):
            files_list.append(os.path.join(root, file))

files_normal = random.sample(files_list,10000 ) 

destPath = r"C:\Users\Matrix BR\Dataset\Smaller_negative"  
 
if os.path.isdir(destPath) == False:
        os.makedirs(destPath)

# iteraate over all random files and move them
for file in files_normal:
    shutil.copy(file, destPath)
    
print(len(files_normal))

#End